# DTW with recorded coordinates

In [1]:
pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-py3-none-any.whl size=3566 sha256=5aa19b75247b7ad0adf0c49ac3c15c77bffdcf8319bcfec08906c3f42074c092
  Stored in directory: /Users/Soohwan/Library/Caches/pip/wheels/e9/ac/30/c962f9d759dd68cb5482727c44441fdfb48040fdbe983857e8
Successfully built fastdtw
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time

In [ ]:
import numpy as np
import cv2
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# assume video_path is the path to the video file
# video_path = "path/to/video.mp4"
video_path = '../../data/videos/'
sampleVideo = video_path + 'sampleVideo.mp4'
basicFour = video_path + 'basic_four.mp4'
basicTwo = video_path + 'basic_two.mp4'
swingDiff = video_path + 'swing_diff.mp4'
swingSame = video_path + 'swing_same.mp4'
swingRaw = video_path + 'swingRaw.mp4'
temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
temporalEasy = video_path + 'temporalEasyRaw.mp4'

# assume pose1_coords is a numpy array of shape (M, 2),
# where M is the number of pose keypoints * 2
pose1_coords = None

# read the video using OpenCV
cap = cv2.VideoCapture(swingSame)

while cap.isOpened():
    # read a frame from the video
    ret, frame = cap.read()

    if not ret:
        break

    # assume pose estimation code extracts the pose coordinates from the frame
    pose2 = pose_estimation(frame)

    if pose1_coords is None:
        # initialize the reference pose sequence with the first pose
        pose1_coords = pose2.flatten()
        continue

    # compute the Euclidean distance between corresponding keypoints
    distance = lambda a, b: euclidean(a, b)

    # compute the DTW distance and path
    distance, path = fastdtw(pose1_coords.reshape(1, -1), pose2.flatten().reshape(1, -1), dist=distance)

    # compute the similarity between the poses (using a normalized distance)
    similarity = 1.0 / (1.0 + distance)

    # compute the time difference between the current frame and the reference frame
    time_diff = (path[-1][0] - path[0][0]) / float(len(pose1_coords))

    # update the reference pose sequence with the current pose
    pose1_coords = pose2.flatten()

    # print the results
    print("Pose similarity: ", similarity)
    print("Time difference: ", time_diff)